## WEBSCRAPING DE GENECARDS

GRUPO 7:
 - Salvador Cama, Alejandro 20190310
 - Velasquez Gonzales, Pedro Daniel 20170382
 - Farfán Florián Erik David 20160229
 - Inoñan Sandoval, José Agustín  20190299

Este archivo necesita un ejecutable que debe ser descargado del siguiente link:
 - https://chromedriver.chromium.org/downloads

La ruta en donde se colocará el ejecutable debe ser de fácil acceso, ya que será utilizada en una de las siguientes líneas de código.

In [14]:
# Importamos las librerías necesarias para realizar el webscraping
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import numpy as np
import xlwings as xw
import xlrd 

In [3]:
# Extraemos los datos del archivo Excel
datos=pd.read_excel("CHGenesOrdenadosConVecinos.xlsx",engine="openpyxl",skiprows=1,usecols=[0,1,2,3,4,5,6,7])
# Imprimimos los datos extraídos
datos


,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
207,PLCL2,NaN,NaN,153722829,153936148,NaN,NaN,NaN
208,TBC1D5,NaN,NaN,153936559,154524620,NaN,NaN,NaN
209,TRNAW-CCA,NaN,NaN,154244176,154244247,NaN,NaN,NaN
210,SATB1,NaN,NaN,155262593,155364610,NaN,NaN,NaN


In [4]:
# Creando una lista con los nombres abreviados de los genes 
lista_genes = datos['Gen Abrev'].values
print(lista_genes)
# print(len(lista_genes))   # son 212 genes

['TRNAS-GGA' 'LOC102173449' 'LOC102169333' 'LOC108636757' 'LOC102187712'
 'VGLL3' 'CHMP2B' 'POU1F1' 'HTR1F' 'LOC102188976' 'CGGBP1' 'ZNF654'
 'C1H3orf38' 'LOC102190548' 'SLC12A8' 'LOC106502043' 'ZNF148' 'SNX4'
 'OSBPL11' 'LMLN' 'RPL35A' 'IQCG' 'LRCH3' 'FYTTD1' 'RUBCN' 'MUC20' 'MUC4'
 'TNK2' 'TFRC' 'ZDHHC19' 'SLC51A' 'PCYT1A' 'TCTEX1D2' 'TM4SF19' 'UBXN7'
 'LOC102173689' 'LOC106501980' 'RNF168' 'SMCO1' 'WDR53' 'FBXO45'
 'LOC102175715' 'NRROS' 'CEP19' 'PIGX' 'PAK2' 'LOC108635601' 'SENP5'
 'NCBP2' 'PIGZ' 'LOC102176365' 'MELTF' 'RTP2' 'SST' 'RTP4' 'LOC102171192'
 'MASP1' 'RTP1' 'LOC102179137' 'ST6GAL1' 'TRNAK-UUU' 'ADIPOQ' 'RFC4'
 'EIF4A2' 'MIR1248' 'KNG1' 'HRG' 'FETUB' 'AHSG' 'LOC108635832' 'DNAJB11'
 'TBCCD1' 'CRYGS' 'DGKG' 'ETV5' 'LOC102179418' 'LOC108635834'
 'LOC102178398' 'IGF2BP2' 'SENP2' 'POLR2H' 'CLCN2' 'LOC106502122'
 'FAM131A' 'EIF4G1' 'PSMD2' 'ECE2' 'CAMK2N2' 'ALG3' 'MIR1224' 'ABCF3'
 'AP2M1' 'DVL3' 'EIF2B5' 'LOC102185396' 'LOC102185870' 'LOC102183010'
 'LOC108636852' 'LOC102183

In [8]:
# Generamos los link`s correspondientes a cada gen y los almacenamos en una lista.
links_genes=[]
# Recorremos la lista de genes para crear los urls necesarios
for i in lista_genes:
    url="https://www.genecards.org/cgi-bin/carddisp.pl?gene="+i+"#expression"
    links_genes.append(url)  #llenamos la lista con cada url
print(links_genes)


['https://www.genecards.org/cgi-bin/carddisp.pl?gene=TRNAS-GGA#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102173449#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102169333#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC108636757#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102187712#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=VGLL3#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=CHMP2B#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=POU1F1#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=HTR1F#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=LOC102188976#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=CGGBP1#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=ZNF654#expression', 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=C1H3orf38#expression', 'https://www.genecards.org/cg

In [9]:
# WARNING: Tener paciencia, demorará un poco
# Usamos webdriver para controlar chrome
driver=webdriver.Chrome(executable_path="C:/Users/RYZEN 5-2600/Documents/chromedriver.exe")
# CREAMOS LISTA VACIA html
html=[]
# Recorremos la lista de links de los genes
for link in links_genes:
    driver.get(link) #abre cada link
    try:            #evita errores
        element = driver.find_element_by_id("expressionImage") #recurre al motor de busqueda
        soup = BeautifulSoup(driver.page_source,features="html.parser") #crea la sopa
        while(soup.find("area")==None): #mientras area sea None, llenamos la lista html
            driver.get(link)
            soup = BeautifulSoup(driver.page_source,features="html.parser")
        html.append(soup)
    except:
        html.append(None)
        continue
driver.quit() #cerremos el motor de busqueda

In [11]:
#Se crea una lista vacia llamada info
info = [] 
for i in html:
    L=[]  # lista vacía para almacenar los datos solicitados de los html`s
    if i == None:   # si no encuentra el html lo almacena como None
        L.append(None)
    else:  
        soup=i  # de lo contrario, si encuentra un html con informacion se comienza a agragar los datos solicitados utilizando BeautifulSoup
        L.append(soup.find('span',{'class':'gc-category'}).text)
        L.append(soup.find_all('span',{'class':'aliasMainName'})[1].text)
        lista=soup.find_all('area',{'title':re.compile('.*Breast.*')})
        L.append(lista[0]['title'].split(',')[0]+', '+lista[3]['title'].split(',')[0])
        L.append(lista[1]['title'].split(',')[0])
        L.append(lista[2]['title'].split(',')[0])
    info.append(L)

print(info)

[[None], [None], [None], [None], [None], ['(Protein Coding)', 'Vestigial Like Family Member 3', '25, 44', 'Breast - No Data', '0'], ['(Protein Coding)', 'Charged Multivesicular Body Protein 2B', '42, 44', 'Breast - No Data', '21'], ['(Protein Coding)', 'POU Class 1 Homeobox 1', '2, 0', 'Breast - No Data', '3'], ['(Protein Coding)', '5-Hydroxytryptamine Receptor 1F', '6, 3', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'CGG Triplet Repeat Binding Protein 1', '43, 49', 'Breast - No Data', '18'], ['(Protein Coding)', 'Zinc Finger Protein 654', '23, 20', 'Breast - No Data', '0'], [None], [None], ['(Protein Coding)', 'Solute Carrier Family 12 Member 8', '10, 11', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'Zinc Finger Protein 148', '22, 26', 'Breast - No Data', '3'], ['(Protein Coding)', 'Sorting Nexin 4', '35, 38', 'Breast - No Data', '6'], ['(Protein Coding)', 'Oxysterol Binding Protein Like 11', '22, 27', 'Breast - No Data', 'Breast - N

In [12]:
# Creamos el data frame "info2" con las columnas que poseen la informacion requerida
info2 = pd.DataFrame(info , columns=['Category','Name of the gene','RNASeq','Microarray','SAGE'])
# Creamos la primera columna con los nombres de los genes
columna1=datos['Gen Abrev']
# Creamos las columnas de categoria y nombre
columna2=info2.iloc[:,[0,1]]
# Creamos las columnas con los datos extraidos del excel 
columna3=datos[['Inicio','Final']]
# Creamos las columnas con la informacion del BREAST
columna4=info2.iloc[:,2:5]
# Concatenamos las columnas 
EXCEL=pd.concat([columna1,columna2,columna3,columna4],axis=1)
# Llenamos con NULL los casilleros vacios
EXCEL=EXCEL.fillna(value='Null')
# Imprimimos las primeros 20 filas
EXCEL.head(20)

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,Null,Null,32091689,32091761,Null,Null,Null
1,LOC102173449,Null,Null,32837131,32838076,Null,Null,Null
2,LOC102169333,Null,Null,33089112,33090132,Null,Null,Null
3,LOC108636757,Null,Null,33764252,33768221,Null,Null,Null
4,LOC102187712,Null,Null,33810174,33811741,Null,Null,Null
5,VGLL3,(Protein Coding),Vestigial Like Family Member 3,33882149,33934641,"25, 44",Breast - No Data,0
6,CHMP2B,(Protein Coding),Charged Multivesicular Body Protein 2B,34199383,34231630,"42, 44",Breast - No Data,21
7,POU1F1,(Protein Coding),POU Class 1 Homeobox 1,34235896,34251973,"2, 0",Breast - No Data,3
8,HTR1F,(Protein Coding),5-Hydroxytryptamine Receptor 1F,34811253,34812505,"6, 3",Breast - No Data,Breast - No Data
9,LOC102188976,Null,Null,34826096,34855603,Null,Null,Null


In [13]:
#Generamos el Excel con los datos ya encontrados y contatenados en EXCEL
EXCEL.to_excel("CHGenesOrdenadosConVecinosPc2.xlsx", index=False)

**BIBLIOGRAFIA:**
 - https://github.com/prbehere93/Genecards_Webscraping/blob/main/Genecards_scraper.ipynb